<a href="https://colab.research.google.com/github/Ezzeldin-nasser939/Word2Vec_LSTM/blob/main/Word2Vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import re
import nltk
import string
import os
import itertools
import numpy as np
import pandas as pd
from pathlib import Path
from keras.models import Model
from keras.layers import Dense
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import LSTM, Dense ,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
def preprocess_text(text):
    # Remove links
    text = re.sub(r"http\S+", "", text)
    # Remove numbers and special characters
    text = re.sub("[^A-Za-z]+", " ", text)
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Convert tokens to lowercase and remove leading/trailing spaces
    tokens = [w.lower().strip() for w in tokens]
    # Return the list of cleaned tokens
    return tokens

In [ ]:
train = []
val = []
files =Path('/content/drive/MyDrive/NLP_Bouns').glob('*')
for myFile in files:
    if (str(myFile)[33:42] == 'enronsent'):
       if (0<= int(str(myFile)[42:]) <=10):
          with open(myFile) as f:
            text = f.read()
          train.append(text)
       if (11<= int(str(myFile)[42:]) <=15):
          with open(myFile) as f:
            text = f.read()
          val.append(text)

In [ ]:
print(len(train))

11


In [ ]:
print(len(val))

5


In [ ]:
all_texts = train + val

In [ ]:
print(len(all_texts))

16


In [ ]:
tokens=[]
for text in all_texts:
    tokens.append(preprocess_text(text))

In [ ]:
import pickle
# with open('/content/drive/MyDrive/NLP_Bouns/tokens.pickle', 'wb') as handle:
#     pickle.dump(tokens, handle, protocol=pickle.HIGHEST_PROTOCOL)


with open('/content/drive/MyDrive/NLP_Bouns/tokens.pickle', 'rb') as handle:
    tokens = pickle.load(handle)

In [ ]:
print(len(tokens))

16


In [ ]:
tokens[0]

['attached',
 'are',
 'two',
 'files',
 'that',
 'illustrate',
 'the',
 'following',
 'as',
 'prices',
 'rose',
 'supply',
 'increased',
 'and',
 'demand',
 'decreased',
 'now',
 'prices',
 'are',
 'beginning',
 'to',
 'fall',
 'in',
 'response',
 'these',
 'market',
 'responses',
 'financial',
 'west',
 'desk',
 'mid',
 'market',
 'share',
 'information',
 'about',
 'yourself',
 'create',
 'your',
 'own',
 'public',
 'profile',
 'at',
 'utility',
 'xls',
 'utility',
 'xls',
 'enron',
 'admin',
 'fsddatasvc',
 'com',
 'on',
 'am',
 'executive',
 'impact',
 'influence',
 'program',
 'immediate',
 'action',
 'required',
 'do',
 'not',
 'delete',
 'as',
 'part',
 'of',
 'the',
 'executive',
 'impact',
 'and',
 'influence',
 'program',
 'each',
 'participant',
 'is',
 'asked',
 'to',
 'gather',
 'input',
 'on',
 'the',
 'participant',
 's',
 'own',
 'management',
 'styles',
 'and',
 'practices',
 'as',
 'experienced',
 'by',
 'their',
 'immediate',
 'manager',
 'each',
 'direct',
 'report'

# Train WordVec

In [ ]:
word2vec = Word2Vec(sentences=tokens, window=5, min_count=5, workers=4)

In [ ]:
# Convert words to word embeddings
word_vectors = word2vec.wv
print(word_vectors)

KeyedVectors<vector_size=100, 22884 keys>


In [ ]:
word2vec

In [ ]:
vocab_size = len(word_vectors.key_to_index)
embedding_size = word2vec.vector_size

In [ ]:
word_vectors.key_to_index

{'the': 0,
 'to': 1,
 'and': 2,
 'of': 3,
 'i': 4,
 'a': 5,
 'in': 6,
 'you': 7,
 'for': 8,
 'that': 9,
 'is': 10,
 'on': 11,
 'this': 12,
 'we': 13,
 'be': 14,
 'have': 15,
 's': 16,
 'with': 17,
 'it': 18,
 'will': 19,
 'are': 20,
 'if': 21,
 'at': 22,
 'as': 23,
 'not': 24,
 'me': 25,
 'or': 26,
 'your': 27,
 'from': 28,
 'enron': 29,
 'by': 30,
 'please': 31,
 'pm': 32,
 'would': 33,
 'can': 34,
 'am': 35,
 'thanks': 36,
 't': 37,
 'was': 38,
 'any': 39,
 'omni': 40,
 'know': 41,
 'an': 42,
 'has': 43,
 'all': 44,
 'do': 45,
 'but': 46,
 'our': 47,
 'my': 48,
 'm': 49,
 'they': 50,
 'e': 51,
 'get': 52,
 'he': 53,
 'what': 54,
 'so': 55,
 'up': 56,
 'about': 57,
 'out': 58,
 'there': 59,
 'let': 60,
 'need': 61,
 'gas': 62,
 'power': 63,
 'd': 64,
 'time': 65,
 'should': 66,
 'com': 67,
 'like': 68,
 'one': 69,
 'been': 70,
 'may': 71,
 'call': 72,
 'new': 73,
 'energy': 74,
 'just': 75,
 'which': 76,
 'some': 77,
 'no': 78,
 'us': 79,
 'also': 80,
 'said': 81,
 'when': 82,
 'these

In [ ]:
len(word_vectors[0])

100

In [ ]:
print(vocab_size)
print(embedding_size)

22884
100


In [ ]:
word2vec.wv['mike']

array([-2.68801242e-01,  4.01468426e-01,  1.20359205e-01,  5.61663844e-02,
        7.08519071e-02, -5.04050672e-01,  2.47982562e-01,  8.97321284e-01,
       -3.61332476e-01, -4.43200439e-01, -3.21669765e-02, -6.38027608e-01,
        2.10816581e-02,  4.28247958e-01,  2.72256732e-01, -2.77095705e-01,
        1.49813101e-01, -3.64839137e-01, -8.47803205e-02, -7.51970649e-01,
        8.30617920e-02,  9.64017808e-02,  4.62772250e-01, -2.68380105e-01,
       -1.92235872e-01,  5.44999763e-02, -3.23558033e-01, -8.59962851e-02,
       -5.72472095e-01,  1.91982642e-01,  6.23424053e-01, -1.42208859e-01,
        3.27955484e-01, -5.56120217e-01,  8.14713817e-03,  2.43139550e-01,
        7.33324662e-02, -1.07383050e-01, -2.45077670e-01, -4.32427347e-01,
        2.49087624e-02, -4.96492893e-01, -1.08165130e-01, -1.67254061e-01,
        2.75016874e-01, -3.21553886e-01, -3.08139086e-01, -6.04518615e-02,
        3.62721980e-01,  2.67538160e-01,  4.04959649e-01, -4.80944872e-01,
        9.61078629e-02, -

In [ ]:
print(len(word2vec.wv['mike']))

100


In [ ]:
word2vec.wv.most_similar(positive=['mike'], topn=10)

[('eric', 0.9985460638999939),
 ('scott', 0.996785581111908),
 ('jim', 0.9952967166900635),
 ('regards', 0.9952590465545654),
 ('thanx', 0.9950001835823059),
 ('x', 0.994477391242981),
 ('steve', 0.9942074418067932),
 ('tara', 0.9940398335456848),
 ('download', 0.993894636631012),
 ('phillip', 0.9936159253120422)]

In [ ]:
seq_length = 5

In [ ]:
def xandy(dataset,seq_length = 5):
    # Initialize empty lists for input and output data
    input_data = []
    output_data = []

    # Loop through each paragraph in the dataset
    for paragraph in dataset:

        # Split the paragraph into words
        words = preprocess_text(paragraph)
        # Loop through each sequence in the paragraph
        for i in range(len(words) - seq_length):

            # Get the input and output sequences
            seq_in = words[i:i + seq_length]
            seq_out = words[i + seq_length]

            # Add the sequences to the input and output lists
            try:
              input_sequence = [word_vectors.key_to_index[token] for token in seq_in]
              output_sequence =word_vectors.key_to_index[seq_out]
            except KeyError:
                # Skip this sequence if a token is not present in the vocabulary
                continue
            # Add the input sequence to the input list
            input_data.append(input_sequence)
            # Add the output sequence to the output list
            output_data.append(output_sequence)

    return  np.array(input_data) ,np.array(output_data)[: ,None]

In [ ]:
X_train,y_train = xandy(train,seq_length)
X_val,y_val = xandy(val,seq_length)

In [ ]:
X_train[0]

array([ 116,   20,  149, 1230,    9])

In [ ]:
X_train.shape

(3446619, 5)

In [ ]:
y_train[0]

array([8124])

In [ ]:
print(embedding_size)

100


In [ ]:
X_train.shape[1]

5

In [ ]:
vocab_size

22355

In [ ]:
# Prepare the LSTM model
sequence_length = seq_length#X_train.shape[1]

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size,input_length=sequence_length, trainable=False))
model.add(LSTM(units=256))
model.add(Dense(units=vocab_size, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            2235500   
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 22355)             5745235   
                                                                 
Total params: 8,346,303
Trainable params: 6,110,803
Non-trainable params: 2,235,500
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=5000, epochs=100, validation_data=(X_val,y_val))

Epoch 1/100
14416/14416 [==============================] - 146s 10ms/step - loss: 7.6749 - accuracy: 0.0642 - val_loss: 8.2089 - val_accuracy: 0.0313
Epoch 2/100
14416/14416 [==============================] - 153s 11ms/step - loss: 7.1347 - accuracy: 0.1063 - val_loss: 8.1282 - val_accuracy: 0.0443
Epoch 3/100
14416/14416 [==============================] - 151s 10ms/step - loss: 6.8169 - accuracy: 0.1276 - val_loss: 8.1168 - val_accuracy: 0.0504
Epoch 4/100
14416/14416 [==============================] - 150s 10ms/step - loss: 6.6042 - accuracy: 0.1433 - val_loss: 8.1362 - val_accuracy: 0.0540
Epoch 5/100
14416/14416 [==============================] - 135s 9ms/step - loss: 6.4556 - accuracy: 0.1549 - val_loss: 8.1373 - val_accuracy: 0.0568
Epoch 6/100
14416/14416 [==============================] - 136s 9ms/step - loss: 6.3488 - accuracy: 0.1633 - val_loss: 8.1927 - val_accuracy: 0.0578
Epoch 7/100
14416/14416 [==============================] - 155s 11ms/step - loss: 6.2690 - accuracy: 0

In [ ]:
model.save('/content/drive/MyDrive/NLP_Bouns/NLP_Bouns.h5')

In [ ]:
model.load_weights('/content/drive/MyDrive/NLP_Bouns/NLP_Bouns (1).h5')

In [ ]:
import random
def predict_next_word(input_word):
    input_sequence = input_word
    try:
        input_sequence = [[word_vectors.key_to_index[word] for word in input_sequence]]
    except KeyError:
        return "not founded"
    input_sequence = pad_sequences(input_sequence, maxlen=sequence_length, padding='post')
    predicted_probs = model.predict(input_sequence)[0]
    predicted_word_index = np.argmax(predicted_probs)
    predicted_word = word_vectors.index_to_key[predicted_word_index]

    return predicted_word

In [ ]:
def complete_sentence():
    sentence = []
    pred = []
    input_word = input("Enter the first word: ")
    sentence.append(input_word)
    pred.append(input_word.lower())
    while True:
        next_word = predict_next_word(pred)
        if next_word == "not founded":
            print("Error Enter another word")
            pred.pop()
            input_word = input("Enter the correct word: ")
            pred.append(input_word)
        else:
            print(f"Is your next word: \"{next_word}\"")
            answer = input("Yes or No? (Enter -1 to terminate): ")
            if answer.lower() == "yes":
                sentence.append(next_word)
                pred.append(next_word.lower())
                input_word = next_word
            elif answer.lower() == "no":
                correct_word = input("Enter the correct word: ")
                pred.append(correct_word.lower())
                sentence.append(correct_word)
                input_word = correct_word
            elif answer == "-1":
                break
            else:
                print("Invalid input. Please try again.")
            if len(pred)>5:
              pred.pop(0)
    completed_sentence = " ".join(sentence)
    print("Your final Sentence is: \n", completed_sentence)

# The First Test case


In [ ]:
#  we want to do this as soon as possible i was prepared to take the package
complete_sentence()
print()

Enter the first word: we
1/1 [==============================] - 0s 30ms/step
Is your next word: "lesser"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: want
1/1 [==============================] - 0s 26ms/step
Is your next word: "desk"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: to
1/1 [==============================] - 0s 31ms/step
Is your next word: "game"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: do
1/1 [==============================] - 0s 30ms/step
Is your next word: "same"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: this
1/1 [==============================] - 0s 28ms/step
Is your next word: "weekend"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: as
1/1 [==============================] - 0s 41ms/step
Is your next word: "soon"
Yes or No? (Enter -1 to terminate): yes
1/1 [==============================] - 0s 40ms/step
Is your next word: "as"
Yes or No? (Enter -1 to terminate): yes
1/1 [=======

# The Seconed Test Case


In [ ]:
# The owners have also agreed to a structure in principle and whereby organizations have determined a that they will sell
complete_sentence()
print()

Enter the first word: The
1/1 [==============================] - 0s 30ms/step
Is your next word: "courtyard"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: owners
1/1 [==============================] - 0s 30ms/step
Is your next word: "state"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: have
1/1 [==============================] - 0s 68ms/step
Is your next word: "px"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: also
1/1 [==============================] - 0s 32ms/step
Is your next word: "blanket"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: agreed
1/1 [==============================] - 0s 30ms/step
Is your next word: "to"
Yes or No? (Enter -1 to terminate): yes
1/1 [==============================] - 0s 33ms/step
Is your next word: "the"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: a
1/1 [==============================] - 0s 31ms/step
Is your next word: "meeting"
Yes or No? (Enter -1 to terminate): no
E

# The Third Test Case

In [ ]:
# We are continuing to work on identifying brown field coal fired sites.
complete_sentence()
print()

Enter the first word: We
1/1 [==============================] - 0s 34ms/step
Is your next word: "lesser"
Yes or No? (Enter -1 to terminate): We
Invalid input. Please try again.
1/1 [==============================] - 0s 33ms/step
Is your next word: "lesser"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: are
1/1 [==============================] - 0s 28ms/step
Is your next word: "lesser"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: continuing
1/1 [==============================] - 0s 30ms/step
Is your next word: "process"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: to
1/1 [==============================] - 0s 29ms/step
Is your next word: "office"
Yes or No? (Enter -1 to terminate): no
Enter the correct word: work
1/1 [==============================] - 0s 26ms/step
Is your next word: "on"
Yes or No? (Enter -1 to terminate): yes
1/1 [==============================] - 0s 28ms/step
Is your next word: "the"
Yes or No? (Enter -1 to terminate)